In [1]:
!pip install tensorflow matplotlib numpy


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Load Fashion MNIST
(train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize to [-1, 1]
BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Generator
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((7, 7, 256)),

        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                      input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    return cross_entropy(tf.ones_like(real_output), real_output) + \
           cross_entropy(tf.zeros_like(fake_output), fake_output)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(f"✅ Epoch {epoch + 1}/{epochs} complete")
    # Save the model after training
    generator.save('your_saved_generator_model.h5')
    print("🎉 Generator model saved as 'your_saved_generator_model.h5'")



C:\Users\A\anaconda3\envs\gpu-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\A\anaconda3\envs\gpu-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
train(train_dataset, EPOCHS)


✅ Epoch 1/50 complete
✅ Epoch 2/50 complete
✅ Epoch 3/50 complete
✅ Epoch 4/50 complete
✅ Epoch 5/50 complete
✅ Epoch 6/50 complete
✅ Epoch 7/50 complete
✅ Epoch 8/50 complete
✅ Epoch 9/50 complete
✅ Epoch 10/50 complete
✅ Epoch 11/50 complete
✅ Epoch 12/50 complete
✅ Epoch 13/50 complete
✅ Epoch 14/50 complete
✅ Epoch 15/50 complete
✅ Epoch 16/50 complete
✅ Epoch 17/50 complete
✅ Epoch 18/50 complete
✅ Epoch 19/50 complete
✅ Epoch 20/50 complete
✅ Epoch 21/50 complete
✅ Epoch 22/50 complete
✅ Epoch 23/50 complete
✅ Epoch 24/50 complete
✅ Epoch 25/50 complete
✅ Epoch 26/50 complete
✅ Epoch 27/50 complete
✅ Epoch 28/50 complete
✅ Epoch 29/50 complete
✅ Epoch 30/50 complete
✅ Epoch 31/50 complete
✅ Epoch 32/50 complete
✅ Epoch 33/50 complete
✅ Epoch 34/50 complete
✅ Epoch 35/50 complete
✅ Epoch 36/50 complete
✅ Epoch 37/50 complete
✅ Epoch 38/50 complete
✅ Epoch 39/50 complete
✅ Epoch 40/50 complete
✅ Epoch 41/50 complete
✅ Epoch 42/50 complete
✅ Epoch 43/50 complete
✅ Epoch 44/50 comple

✅ Epoch 50/50 complete
🎉 Generator model saved as 'your_saved_generator_model.h5'
